In [1]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from algorithms import *
from multiprocessing import Pool, shared_memory

### Make sure the proc_num is around 2/3 of the number of your machine's cores

In [2]:
proc_num = 55

# Sampling off-diagonal terms from a exponential distribution

In [ ]:
# netwrok sizes
ns = np.array([4, 8, 16, 32, 64, 128, 256, 512, 1024])
# maximim degredatopn rate
c = 1
# lambda or the exponential distribution rates of off-diagonal terms
lambs = np.array([ c*10, c*50,  1000*c])
# trials per ensemble
trials = 1000
# number of ensembles
repeats = 5
def proc_stability1(args):
    # initialise each random generator inside the running thread
    rng = np.random.default_rng()
    np.random.seed(rng.integers(low=0, high=np.iinfo(np.int32).max))
    # unbox the parameters
    n, lamb, c, trials, repeat_id = args
    super_stable_count = 0
    stable_count = 0
    for i in range(trials):
        # All the of-diagonal elements are non-zero
        # Also, the off-diagonal rate is scaled by sqrt(n)
        m = sample_matrix(n, 
                          p_diag_uniform(-c), 
                          p_off_diag_exp(lamb))
        region1 = alg1(m, axis=0)
        if region1 == 0:
            super_stable_count += 1
        else:# If in the inconclusive region, check its stability
            if not is_unstable(m):
                stable_count += 1
        
    
    
    return (repeat_id, super_stable_count/trials, stable_count/trials)

# Prepare parameters for each thread
params1 = [ (n, lamb, c, trials, repeat_id) 
           for n in ns for lamb in lambs for repeat_id in range(repeats)]
# Parallelism the sampling
with Pool(proc_num) as pool:
    res1 = pool.map(proc_stability1, params1)
    
np.save("P_I_P_S_expo_no_scaling.npy",res1)

# Sampling off-diagonal terms from a folded-normal distribution

In [ ]:
# netwrok sizes
ns = np.array([4, 8, 16, 32, 64, 128, 256, 512, 1024])
# maximim degredatopn rate
c = 1
# standard deviation of the gaussian distribution of off-diagonal terms
stds = np.array([ 0.01, 0.1, 1])
# trials per ensemble
trials = 1000
# number of ensembles
repeats = 5

def proc_stability2(args):
    # initialise each random generator inside the running thread
    rng = np.random.default_rng()
    np.random.seed(rng.integers(low=0, high=np.iinfo(np.int32).max))
    # unbox the parameters
    n, std, c, trials, repeat_id = args
    super_stable_count = 0
    stable_count = 0
    for i in range(trials):
        # All the diagonal elements are sampled from the uniform distribution U(-c, 0)
        # All the of-diagonal elements are non-zero
        m = sample_matrix(n, 
                          p_diag_uniform(-c), 
                          p_off_diag_normal(0, std))
        region1 = alg1(m, axis=0)        
        if region1 == 0:
            super_stable_count += 1
        else:# If in the inconclusive region, check its 
            if not is_unstable(m):
                stable_count += 1
        
    
    
    return (repeat_id, super_stable_count/trials, stable_count/trials)

# Prepare parameters for each thread
params2 = [ (n, std, c, trials, repeat_id) 
           for n in ns for std in stds for repeat_id in range(repeats)]
# Parallelism the sampling
with Pool(proc_num) as pool:
    res2 = pool.map(proc_stability2, params2)
    
np.save("P_I_P_S_normal_no_scaling.npy",res2)

# Sampling off-diagonal terms from a folded-normal distribution: Effect of changing the degredation maximum from $c=1$ to $c=100$.

In [ ]:
# netwrok sizes
ns = np.array([4, 8, 16, 32, 64, 128, 256, 512, 1024])
# maximim degredatopn rate
c = 100
# standard deviation of the gaussian distribution of off-diagonal terms
stds = np.array([ 0.01, 0.1, 1])
# trials per ensemble
trials = 1000
# number of ensembles
repeats = 5

def proc_stability3(args):
    # initialise each random generator inside the running thread
    rng = np.random.default_rng()
    np.random.seed(rng.integers(low=0, high=np.iinfo(np.int32).max))
    # unbox the parameters
    n, std, c, trials, repeat_id = args
    # Initialise the ensembles statistics
    super_stable_count = 0
    stable_count = 0
    for i in range(trials):
        # All the diagonal elements are sampled from the uniform distribution U(-c, 0)
        # All the off-diagonal elements are non-zero
        m = sample_matrix(n, 
                          p_diag_uniform(-c), 
                          p_off_diag_normal(0, std))
        region1 = alg1(m, axis=0)
        if region1 == 0:
            super_stable_count += 1
        else:# If in the inconclusive region, check its 
            if not is_unstable(m):
                stable_count += 1
        
    
    
    return (repeat_id, super_stable_count/trials, stable_count/trials)

# Prepare parameters for each thread
params3 = [ (n, std, c, trials, repeat_id) 
           for n in ns for std in stds for repeat_id in range(repeats)]
# Parallelism the sampling
with Pool(proc_num) as pool:
    res3 = pool.map(proc_stability3, params3)
    
np.save("P_I_P_S_normal_no_scaling_c_100.npy",res3)

# Sampling off-diagonal terms from a folded-normal distribution with included activation

In [75]:
# netwrok sizes
ns = np.array([4, 6, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 256, 384, 512])
# maximim degredatopn rate
c = 1
# standard deviation of the gaussian distribution of off-diagonal terms
std = 0.01
# Activation shifts
activations = np.array([0.0, 0.001, 0.005, 0.01])
# trials per ensemble
trials = 1000
# number of ensembles
repeats = 5

def proc_stability4(args):
    # initialise each random generator inside the running thread
    rng = np.random.default_rng()
    np.random.seed(rng.integers(low=0, high=np.iinfo(np.int32).max))
    # unbox the parameters
    n, std, activation, c, trials, repeat_id = args
    # Initialise the ensembles statistics
    super_stable_count = 0
    stable_count = 0
    for i in range(trials):
        # All the diagonal elements are sampled from the uniform distribution U(-c, 0)
        # All the of-diagonal elements are non-zero
        m = sample_matrix(n, 
                          p_diag_uniform(-c), 
                          p_off_diag_normal(0, std))
        m += np.eye(n)*activation
        region1 = alg1(m, axis=0)        
        if region1 == 0:
            super_stable_count += 1
        else:# If in the inconclusive region, check its 
            if not is_unstable(m):
                stable_count += 1
        
    
    
    return (repeat_id, super_stable_count/trials, stable_count/trials)

# Prepare parameters for each thread
params4 = [ (n, std, activation, c, trials, repeat_id) 
           for n in ns for activation in activations for repeat_id in range(repeats)]
# Parallelism the sampling
with Pool(proc_num) as pool:
    res4 = pool.map(proc_stability4, params4)
    
np.save("P_I_P_S_normal_activation.npy",res4)

# Sampling off-diagonal terms from a folded-normal distribution with inhibition

In [76]:
# netwrok sizes
ns = np.array([4, 6, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 256, 384, 512])
# maximim degredatopn rate
c = 1
# standard deviation of the gaussian distribution of off-diagonal terms
std = 0.01
# Inhibition shifts
inhibitions = np.array([0.0, 0.001, 0.005, 0.01])
# trials per ensemble
trials = 1000
# number of ensembles
repeats = 5

def proc_stability5(args):
    # initialise each random generator inside the running thread
    rng = np.random.default_rng()
    np.random.seed(rng.integers(low=0, high=np.iinfo(np.int32).max))
    # unbox the parameters
    n, std, inhibition, c, trials, repeat_id = args
    # Initialise the ensembles statistics
    super_stable_count = 0
    stable_count = 0
    for i in range(trials):
        # All the diagonal elements are sampled from the uniform distribution U(-c, 0)
        # All the of-diagonal elements are non-zero
        m = sample_matrix(n, 
                          p_diag_uniform(-c), 
                          p_off_diag_normal(0, std))
        m -= np.eye(n)*inhibition
        region1 = alg1(m, axis=0)
        if region1 == 0:
            super_stable_count += 1
        else:# If in the inconclusive region, check its 
            if not is_unstable(m):
                stable_count += 1
        
    
    
    return (repeat_id, super_stable_count/trials, stable_count/trials)

# Prepare parameters for each thread
params5 = [ (n, std, inhibition, c, trials, repeat_id) 
           for n in ns for inhibition in inhibitions for repeat_id in range(repeats)]
# Parallelism the sampling
with Pool(proc_num) as pool:
    res5 = pool.map(proc_stability5, params5)
    
np.save("P_I_P_S_normal_inhibition.npy",res5)

# Repeat the estimates for random self-activations

In [97]:
# netwrok sizes
ns = np.array([4, 6, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 256, 384, 512])
# maximim degredatopn rate
c = 1
# standard deviation of the gaussian distribution of off-diagonal terms
std = 0.01
# Maximmum activation shifts
activations = np.array([0.0, 0.001, 0.005, 0.01])
# trials per ensemble
trials = 20000
# number of ensembles
repeats = 5

def proc_stability6(args):
    # initialise each random generator inside the running thread
    rng = np.random.default_rng()
    np.random.seed(rng.integers(low=0, high=np.iinfo(np.int32).max))
    # unbox the parameters
    n, std, activation, c, trials, repeat_id = args
    # Initialise the ensembles statistics
    super_stable_count = 0
    stable_count = 0
    for i in range(trials):
        # All the diagonal elements are sampled from the uniform distribution U(-c, 0)
        # All the of-diagonal elements are non-zero
        m = sample_matrix(n, 
                          p_diag_uniform(-c), 
                          p_off_diag_normal(0, std))
        # random shift andom number of activation
        p = np.random.rand(1)
        m += np.diag(np.random.binomial(1, p, n))*np.random.rand(n)*activation
        region1 = alg1(m, axis=0)
        if region1 == 0:
            super_stable_count += 1
        else:# If in the inconclusive region, check its 
            if not is_unstable(m):
                stable_count += 1
        
    
    
    return (repeat_id, super_stable_count/trials, stable_count/trials)

# Prepare parameters for each thread
params6 = [ (n, std, activation, c, trials, repeat_id) 
           for n in ns for activation in activations for repeat_id in range(repeats)]
# Parallelism the sampling
with Pool(proc_num) as pool:
    res6 = pool.map(proc_stability6, params6)
    
np.save("P_I_P_S_normal_activation_random.npy",res6)

# Repeat the estimates for random self-inhibition

In [98]:
# netwrok sizes
ns = np.array([4, 6, 8, 12, 16, 24, 32, 48, 64, 96, 128, 192, 256, 384, 512])
# maximim degredatopn rate
c = 1
# standard deviation of the gaussian distribution of off-diagonal terms
std = 0.01
# Maximmum trial shifts
inhibitions = np.array([0.0, 0.001, 0.005, 0.01])
# trials per ensemble
trials = 20000
# number of ensembles
repeats = 5

def proc_stability7(args):
    # initialise each random generator inside the running thread
    rng = np.random.default_rng()
    np.random.seed(rng.integers(low=0, high=np.iinfo(np.int32).max))
    # unbox the parameters
    n, std, inhibition, c, trials, repeat_id = args
    # Initialise the ensembles statistics
    super_stable_count = 0
    stable_count = 0
    for i in range(trials):
        # All the diagonal elements are sampled from the uniform distribution U(-c, 0)
        # All the of-diagonal elements are non-zero
        m = sample_matrix(n, 
                          p_diag_uniform(-c), 
                          p_off_diag_normal(0, std))
        # random shift andom number of activation
        p = np.random.rand(1)
        m -= np.diag(np.random.binomial(1, p, n))*np.random.rand(n)*inhibition
        region1 = alg1(m, axis=0)
        if region1 == 0:
            super_stable_count += 1
        else:# If in the inconclusive region, check its 
            if not is_unstable(m):
                stable_count += 1
        
    
    
    return (repeat_id, super_stable_count/trials, stable_count/trials)

# Prepare parameters for each thread
params7 = [ (n, std, inhibition, c, trials, repeat_id) 
           for n in ns for inhibition in inhibitions for repeat_id in range(repeats)]
# Parallelism the sampling
with Pool(proc_num) as pool:
    res7 = pool.map(proc_stability7, params7)
    
np.save("P_I_P_S_normal_inhibition_random.npy",res7)